# Analyzing OD Flows for the Boston Area

## Includes Boston, Cambridge and Sommerville


In [474]:
import pandas as pd
import numpy as np
import folium
#from folium.plugins.mouse_position import MousePosition
import math
from collections import namedtuple
from osmnx.utils import get_bearing
import datetime as dt

pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 400)

### First load all BlueBike trips for a given month

All july trips:             (46648, 35575, 63864) -> total = 242889

All working day July trips: (42379, 22439, 49880)

In [475]:
#trips = pd.read_csv("../data/boston/bike-trips/201807-bluebikes-tripdata.csv") # July trips

trips = pd.read_csv("./2016/201610.csv")  # July trips




trips = trips[:50000]




trips = trips[trips['start station latitude'] != 0] # drop 2 rows that have zero latitude
#trips = trips[trips['end station id'] != 231] # let's drop this station that has no trips starting at it

trips2 = trips

total_trips = trips.shape[0]
total_trips

50000

In [476]:
trips

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,623,2016-10-01 00:02:59,2016-10-01 00:13:23,189,Kendall T,42.362428,-71.084955,53,Beacon St / Mass Ave,42.350851,-71.089886,57,Subscriber,1993,2
1,260,2016-10-01 00:03:40,2016-10-01 00:08:01,70,Harvard Kennedy School at Bennett St / Eliot St,42.371992,-71.122270,149,Harvard University Transportation Services - 1...,42.363796,-71.129164,493,Subscriber,1983,1
2,228,2016-10-01 00:04:15,2016-10-01 00:08:03,21,Prudential Center / Belvidere,42.345959,-71.082578,5,Northeastern U / North Parking Lot,42.341814,-71.090179,1865,Subscriber,1991,1
3,1364,2016-10-01 00:05:37,2016-10-01 00:28:22,189,Kendall T,42.362428,-71.084955,36,Boston Public Library - 700 Boylston St.,42.349673,-71.077303,524,Subscriber,1994,1
4,1328,2016-10-01 00:07:52,2016-10-01 00:30:00,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075,22,South Station - 700 Atlantic Ave.,42.352175,-71.055547,1525,Subscriber,1992,1
5,452,2016-10-01 00:11:58,2016-10-01 00:19:31,49,Stuart St. at Charles St.,42.351146,-71.066289,25,Tremont St / W Newton St,42.341332,-71.076847,707,Subscriber,1990,2
6,1302,2016-10-01 00:12:12,2016-10-01 00:33:54,41,Packard's Corner - Comm. Ave. at Brighton Ave.,42.352261,-71.123831,41,Packard's Corner - Comm. Ave. at Brighton Ave.,42.352261,-71.123831,181,Customer,\N,0
7,1140,2016-10-01 00:15:23,2016-10-01 00:34:23,41,Packard's Corner - Comm. Ave. at Brighton Ave.,42.352261,-71.123831,41,Packard's Corner - Comm. Ave. at Brighton Ave.,42.352261,-71.123831,1803,Customer,\N,0
8,181,2016-10-01 00:17:15,2016-10-01 00:20:17,118,Linear Park - Mass. Ave. at Cameron Ave.,42.397828,-71.130516,100,Davis Square,42.396969,-71.123024,1074,Subscriber,1991,2
9,293,2016-10-01 00:22:24,2016-10-01 00:27:17,53,Beacon St / Mass Ave,42.350851,-71.089886,67,MIT at Mass Ave / Amherst St,42.358100,-71.093198,208,Subscriber,1996,1


In [477]:
#pd.set_option('display.max_rows',2000000000000000)
trips.head(300)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,623,2016-10-01 00:02:59,2016-10-01 00:13:23,189,Kendall T,42.362428,-71.084955,53,Beacon St / Mass Ave,42.350851,-71.089886,57,Subscriber,1993,2
1,260,2016-10-01 00:03:40,2016-10-01 00:08:01,70,Harvard Kennedy School at Bennett St / Eliot St,42.371992,-71.122270,149,Harvard University Transportation Services - 1...,42.363796,-71.129164,493,Subscriber,1983,1
2,228,2016-10-01 00:04:15,2016-10-01 00:08:03,21,Prudential Center / Belvidere,42.345959,-71.082578,5,Northeastern U / North Parking Lot,42.341814,-71.090179,1865,Subscriber,1991,1
3,1364,2016-10-01 00:05:37,2016-10-01 00:28:22,189,Kendall T,42.362428,-71.084955,36,Boston Public Library - 700 Boylston St.,42.349673,-71.077303,524,Subscriber,1994,1
4,1328,2016-10-01 00:07:52,2016-10-01 00:30:00,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075,22,South Station - 700 Atlantic Ave.,42.352175,-71.055547,1525,Subscriber,1992,1
5,452,2016-10-01 00:11:58,2016-10-01 00:19:31,49,Stuart St. at Charles St.,42.351146,-71.066289,25,Tremont St / W Newton St,42.341332,-71.076847,707,Subscriber,1990,2
6,1302,2016-10-01 00:12:12,2016-10-01 00:33:54,41,Packard's Corner - Comm. Ave. at Brighton Ave.,42.352261,-71.123831,41,Packard's Corner - Comm. Ave. at Brighton Ave.,42.352261,-71.123831,181,Customer,\N,0
7,1140,2016-10-01 00:15:23,2016-10-01 00:34:23,41,Packard's Corner - Comm. Ave. at Brighton Ave.,42.352261,-71.123831,41,Packard's Corner - Comm. Ave. at Brighton Ave.,42.352261,-71.123831,1803,Customer,\N,0
8,181,2016-10-01 00:17:15,2016-10-01 00:20:17,118,Linear Park - Mass. Ave. at Cameron Ave.,42.397828,-71.130516,100,Davis Square,42.396969,-71.123024,1074,Subscriber,1991,2
9,293,2016-10-01 00:22:24,2016-10-01 00:27:17,53,Beacon St / Mass Ave,42.350851,-71.089886,67,MIT at Mass Ave / Amherst St,42.358100,-71.093198,208,Subscriber,1996,1


In [478]:
#boundary limits for the maps
west_limit = -71.166491 - 0.003
east_limit = -71.006098 + 0.0001
north_limit = 42.406302 + 0.003
south_limit = 42.30 

In [479]:
stations = pd.DataFrame()
stations['id']    = trips['start station id']
stations['lat']   = trips['start station latitude']
stations['lon']   = trips['start station longitude']
stations['sname'] = trips['start station name'].str.replace("'", "&#8217;")

stations['id']    = trips['end station id']
stations['lat']   = trips['end station latitude']
stations['lon']   = trips['end station longitude']
stations['sname'] = trips['end station name'].str.replace("'", "&#8217;")

stations.drop_duplicates(subset='id', inplace=True)

stations.set_index('id', inplace=True)
stations.sort_index(inplace=True)
stations.head(200)

,lat,lon,sname
id,,,
1,42.387151,-71.075978,18 Dorrance Warehouse
3,42.340021,-71.100812,Colleges of the Fenway
4,42.345392,-71.069616,Tremont St. at Berkeley St.
5,42.341814,-71.090179,Northeastern U / North Parking Lot
6,42.361174,-71.065142,Cambridge St. at Joy St.
7,42.353287,-71.044389,Fan Pier
8,42.353334,-71.137313,Union Square - Brighton Ave. at Cambridge St.
9,42.351246,-71.115639,Agganis Arena - 925 Comm Ave.
10,42.350406,-71.108279,B.U. Central - 725 Comm. Ave.


In [480]:
 od_matrix_by_station = pd.crosstab(trips['start station id'],trips['end station id'])

## Origin X Destination matrix

In [481]:
def populate_od_by_station (od_matrix_by_station):
    od_matrix_by_station = pd.crosstab(trips['start station id'],trips['end station id'])

In [482]:
populate_od_by_station(od_matrix_by_station)

In [483]:
od_matrix_by_station.head(500)

end station id,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,29,30,31,32,33,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,58,59,60,61,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,84,85,87,88,89,90,91,92,93,94,95,96,97,98,99,100,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,124,125,126,128,129,130,131,133,134,135,136,137,138,139,140,141,142,143,145,146,149,150,151,152,159,160,161,162,163,167,169,170,171,173,174,175,176,177,178,179,180,183,184,186,189,190,192,193,194,195,196,197,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218
start station id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,2,0,3,1,0,0,0,0,5,4,2,7,0,3,1,3,0,4,2,1,0,4,2,2,0,1,0,2,0,2,3,0,3,0,1,0,0,1,4,9,1,1,0,0,1,1,3,0,12,0,5,0,0,1,0,0,0,4,1,3,5,0,0,0,0,3,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,7,1,0,0,0,1,0,0,2,1,0,0,0,0,0,0,0,0,1,7,0,1,2,0,7,0,3,1,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,1,0,3,0,0,0,0,0,0,0,0,0,0,1,5,2,1,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2
4,0,3,4,1,2,2,1,3,2,1,0,11,3,0,7,0,0,2,12,36,11,3,10,2,1,0,0,2,1,0,18,14,0,5,1,0,3,1,2,5,6,0,7,5,5,1,2,3,8,2,1,11,16,3,1,4,2,2,0,1,0,0,0,1,0,0,3,0,1,0,0,0,0,3,0,0,0,0,0,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,5,3,0,0,1,0,0,0,0,0,0,2,0,2,0,2,1,0,0,0,0,0,0,0,1,1,13,0,1,4,0,1,2,0,0,0,3,0,0,0,1,0,1,0,0,0,0,2,1,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
5,0,9,0,15,0,0,6,1,1,9,1,3,5,0,5,0,8,1,8,5,5,0,1,1,4,0,13,0,13,3,0,16,0,6,0,11,1,1,0,20,6,0,1,4,1,2,6,5,4,2,0,0,1,2,12,0,4,8,5,2,3,6,0,0,2,0,1,0,0,0,0,0,4,2,13,0,0,0,0,1,2,2,0,0,0,0,0,3,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,1,5,0,0,3,0,3,0,2,0,0,0,0,0,4,7,0,0,1,0,0,1,11,2,0,0,0,0,0,0,0,1,0,2,0,5,0,3,0,0,0,0,1,3,0,4,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,1,1,1,5,6,0,1,1,0,3,0,1,0,6,0,2,4,1,8,10,0,3,1,0,2,0,0,0,5,6,3,0,2,2,3,8,25,3,3,1,8,16,4,3,0,11,8,9,1,0,6,2,14,5,0,3,0,0,5,3,2,1,1,16,5,3,2,3,1,1,2,9,17,0,6,9,0,3,0,13,6,0,0,0,0,7,1,3,2,0,0,1,0,4,0,1,0,1,0,0,0,0,0,0,5,5,0,0,6,1,0,2,0,3,0,0,0,1,0,6,0,7,0,0,0,7,10,0,4,0,0,1,2,3,2,0,1,0,0,0,0,2,0,1,0,0,0,0,1,5,2,0,0,0,0,4,2,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,5,0,3,0,0,0,0,0,1,1,0,0,1,0,0,10,0,13,17,0,6,10,0,0,0,0,0,1,2,1,0,0,6,0,0,1,2,0,1,11,5,0,2,0,0,1,3,0,0,0,1,3,2,0,3,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,1,2,1,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,5,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,24,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
8,0,0,0,8,0,0,1,6,16,3,1,0,1,2,0,0,1,0,2,0,0,0,0,0,0,3,0,0,14,14,0,1,3,0,0,4,4,1,0,0,0,0,0,0,4,0,1,0,0,1,0,0,0,0,1,0,0,0,8,1,2,1,0,1,0,0,7,0,4,0,0,0,3,1,3,0,0,0,0,2,1,1,0,0,0,0,1,2,0,0,1,0,2,0,0,0,0,6,0,8,0,0,0,0,2,1,0,0,0,0,0,0,0,6,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,0,0,0,0,3,4,2,0,1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,7,1,0,0,0,0,0,0,0,0,0,0
9,0,1,0,2,0,0,13,7,31,1,1,4,4,0,4,1,17,0,1,2,0,0,1,0,0,1,0,0,6,32,1,9,1,2,0,12,3,0,0,4,0,0,0,0,2,1,5,9,2,3,0,0,0,2,13,0,0,0,5,10,15,13,2,0,1,3,3,2,0,0,0,0,8,0,8,0,0,2,1,0,1,3,0,0,0,0,2,8,0,0,0,0,4,0,11,0,5,0,5,5,0,0,0,0,0,1,4,0,0,1,0,5,0,0,0,1,0,0,0,0,1,0,0,0,0,2,0,0,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,1,0,0,3,1,0,7,2,1,1,0,0,4,3,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,1,0,2,0,14,43,13,0,0,0,2,0,2,0,6,1,1,0,0,0,1,0,1,0,1,0,5,22,0,3,3,3,0,33,3,0,1,0,3,0,0,0,1,1,2,11,2,1,0,5,0,5,3,0,0,0,10,11,6,7,7,2,0,2,8,6,3,0,3,0,5,

### Looking at different day periods

# Test this block of code

trips_new = trips.copy()

trips_new['hour'] = trips2['starttime'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S.%f').hour)
trips_new['week_day'] = trips2['starttime'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S.%f').weekday())
#Monday is 0 and Sunday is 6
trips_new['weekend'] = trips_new['week_day'] >= 5

#let's limit our analysis for working days.
trips_morning   = trips_new[(trips_new['hour'] >= 7)  & (trips_new['hour'] <= 9)  & (~trips_new['weekend'])]#morning rush hour (up to 9:59:59)
trips_lunch     = trips_new[(trips_new['hour'] >= 11) & (trips_new['hour'] <= 13) & (~trips_new['weekend'])]#3h lunch time
trips_afternoon = trips_new[(trips_new['hour'] >= 17) & (trips_new['hour'] <= 19) & (~trips_new['weekend'])]#3h afternoon rush hour

od_morning   = pd.DataFrame(0, index=range(len(stations)*len(stations)), columns=range(len(stations)*len(stations)))
od_lunch     = pd.DataFrame(0, index=range(len(stations)*len(stations)), columns=range(len(stations)*len(stations)))
od_afternoon = pd.DataFrame(0, index=range(len(stations)*len(stations)), columns=range(len(stations)*len(stations)))

trips_morning.apply(populate_od_by_station, args=(od_morning,), axis=1) #the comma after od_matrix converts it to a tuple
trips_lunch.apply(populate_od_by_station, args=(od_lunch,), axis=1) 
trips_afternoon.apply(populate_od_by_station, args=(od_afternoon,), axis=1) 

trips_morning.shape[0], trips_lunch.shape[0], trips_afternoon.shape[0]


# Test this as well

od_morning.values.max(), od_lunch.values.max(), od_afternoon.values.max()

In [484]:
def draw_arrow(folium_map, lat1, lon1, lat2, lon2, color='blue', weight = 3, tip=6, text=''):
    '''
    Draws an arrow from lat1lon1 to a little before lat2lon2. The size of the arrow is 95% of the distance between
    the two points. This helps avoiding cluttering the tip of multiple arrows to the same destination
    
    Parameters
    folium_map:
    lat1, lon1: start of the arrow
    lat2, lon2: end of the arrow 
    color: color of the arrow body and tip
    weight: width of the arrow
    tip: size of the tip
    text: option tooltip for the arrow body
    
    Return
    nothing
    '''
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    p1 = Point (.85*lat1 + .15*lat2, .85*lon1 + .15*lon2)
    p2 = Point (.95* lat2 + .05*lat1, .95*lon2 + .05*lon1)
    folium.PolyLine(locations=[p1, p2], color=color, weight = weight, popup=text).add_to(folium_map)
    rotation = get_bearing(p1, p2) - 90
    folium.RegularPolygonMarker(location=p2, fill_color=color, number_of_sides=3, radius=tip, 
                            rotation=rotation, popup=text).add_to(folium_map)

In [485]:
#TEST CODE
def draw_arrow2(folium_map, lat1, lon1, lat2, lon2, color='red', weight = 3, tip=6, text=''):
    '''
    Draws an arrow from lat1lon1 to a little before lat2lon2. The size of the arrow is 95% of the distance between
    the two points. This helps avoiding cluttering the tip of multiple arrows to the same destination
    
    Parameters
    folium_map:
    lat1, lon1: start of the arrow
    lat2, lon2: end of the arrow 
    color: color of the arrow body and tip
    weight: width of the arrow
    tip: size of the tip
    text: option tooltip for the arrow body
    
    Return
    nothing
    '''
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    p1 = Point (.85*lat1 + .15*lat2, .85*lon1 + .15*lon2)
    p2 = Point (.95* lat2 + .05*lat1, .95*lon2 + .05*lon1)
    folium.PolyLine(locations=[p1, p2], color=color, weight = weight, popup=text).add_to(folium_map)
    rotation = get_bearing(p1, p2) - 90
    folium.RegularPolygonMarker(location=p2, fill_color=color, number_of_sides=3, radius=tip, 
                            rotation=rotation, popup=text).add_to(folium_map)

In [486]:
def draw_stations(folium_map):
    for index, row in stations.iterrows():
        folium.CircleMarker(location=[row.lat, row.lon], radius=2,
                            popup=row.sname, color='black').add_to(folium_map)

In [487]:
def flow_map(od_matrix, minimum = -1, show = 4, file_to_save = ""):
    '''
    Creates a flow map based on the given list of trips.
    
    Parameters
    od_matrix: origin-destination matrix count for the regions of the city
    minimum: only draw arrows for the flows that are larger than this minimum number of trips
    show: a measure of the portion of flows to show. Typically between 2 and 5. 
    file_to_save: if not null, save the map in this file
    Return
    a folium map with the arrows representing bike flows drawn
    '''
    fmap = folium.Map([(north_limit+south_limit)/2, (west_limit+east_limit)/2], zoom_start = 13, control_scale = True)
    np.fill_diagonal(od_matrix.values, 0) # ignore round-trips to the same station
    
    maximum = od_matrix.values.max()
    if minimum == -1:
        minimum = maximum / show
    rows, cols = np.where(od_matrix>minimum)

    shown_trips = 0
   
    total_trips = od_matrix.values.sum()
    
    draw_stations(fmap)

    for k in range(rows.size):
        origin_cell = rows[k]
        destination_cell = cols[k]
        num_trips = od_matrix.iloc[origin_cell, destination_cell]
        shown_trips += num_trips
        weight = math.ceil( (num_trips-minimum)/maximum * 10)
   
        o1, o2 = cell_center(origin_cell%GRID_SIZE, origin_cell//GRID_SIZE)
        d1, d2 = cell_center(destination_cell%GRID_SIZE, destination_cell//GRID_SIZE)
        draw_arrow(fmap, o1, o2, d1, d2, 
                   text=str(num_trips) + ' bike trips',
                   weight = weight)   
        
    #fmap.add_child(MousePosition()) # not yet available in the stable release

    folium.Marker(location=cell_center(int(GRID_SIZE/3), int(GRID_SIZE/3)),
                  popup='Showing '+ str(shown_trips)+' trips representing ' + 
                         str(int(100*shown_trips/total_trips)) + '% of all flows'
                 ).add_to(fmap)

    
    if file_to_save:
        fmap.save(file_to_save)
    return fmap

In [492]:
def flow_map_by_station(od_matrix_by_station, minimum = -1, show = 4, file_to_save = ""):
    '''
    Creates a flow map based on the given list of trips.
    
    Parameters
    od_matrix_by_station: origin-destination matrix count for all bike station of the city
    minimum: only draw arrows for the flows that are larger than this minimum number of trips
    show: a measure of the portion of flows to show. Typically between 2 and 5. 
    file_to_save: if not null, save the map in this file
    Return
    a folium map with the arrows representing bike flows drawn
    '''
    fmap = folium.Map([(north_limit+south_limit)/2, (west_limit+east_limit)/2], zoom_start = 13, control_scale = True)
    np.fill_diagonal(od_matrix_by_station.values, 0) # ignore round-trips to the same station
    
    maximum = od_matrix_by_station.values.max()
    if minimum == -1:
        minimum = maximum / show
    rows, cols = np.where(od_matrix_by_station>minimum)

    shown_trips = 0
    shown_trips2 = 0
    total_trips = od_matrix_by_station.values.sum()
    
    draw_stations(fmap)

    stations_list = od_matrix_by_station.columns.values
    
    sum1 = 0
    
    sum2 = 0
    
    for start_id, row in od_matrix_by_station.iterrows():
        for end_id in stations_list:
            number_of_trips = row[end_id]
    #        print(number_of_trips)
            if number_of_trips >= minimum:
                start_latitude = stations.loc[start_id,'lat']
                start_latitude2 = stations.loc[end_id,'lat']
                start_longitude = stations.loc[start_id,'lon']
                start_longitude2 = stations.loc[end_id,'lon']
                end_latitude = stations.loc[end_id,'lat']
                end_latitude2 = stations.loc[start_id,'lat']
                end_longitude = stations.loc[end_id,'lon']
                end_longitude2 = stations.loc[start_id,'lon']
                num_trips = od_matrix_by_station.loc[start_id, end_id]
                            #print(num_trips)
                num_trips2 = od_matrix_by_station.loc[end_id, start_id]
                            #print(num_trips2)
                shown_trips += num_trips
                shown_trips2 += num_trips2
                weight = math.ceil( (num_trips-minimum)/maximum * 10)             #change this number for arrow thickness
                weight2 = math.ceil( (num_trips2-minimum)/maximum * 10)
                            #print(start_latitude,start_longitude,end_latitude,end_longitude,num_trips,shown_trips,weight)
                            #if (num_trips == num_trips2) or (shown_trips == shown_trips2):
                        
                        
                        
                if abs(num_trips - num_trips2) > 40:
                    
                    
                    
                    
                    draw_arrow(fmap, start_latitude2, start_longitude2,end_latitude2,end_longitude2, text=str(num_trips2) + ' bike trips', weight = weight)
                    draw_arrow2(fmap, start_latitude, start_longitude,end_latitude,end_longitude, text=str(num_trips) + ' bike trips', weight = weight)
                            #sum1 = num_trips
                            #print(sum1)
                    diff = abs(num_trips - num_trips2)
                    print("Difference between %s and %s is %s" %(num_trips, num_trips2, diff))
                    print(stations.loc[stations['lat'] == start_latitude, 'sname'].item())                     
                    print(stations.loc[stations['lat'] == end_latitude, 'sname'].item())                       

                    print("____________________________________")

    
    folium.Marker(location=[(north_limit+south_limit)/2,(west_limit+east_limit)/2],
                  popup='Showing '+ str(shown_trips)+' trips representing ' + 
                         str(int(100*shown_trips/total_trips)) + '% of all flows'
                 ).add_to(fmap)
    
    if file_to_save:
        fmap.save(file_to_save)
    return fmap

In [493]:
flow_map_by_station(od_matrix_by_station, minimum=2, file_to_save="boston_station_flow_alldays_201610.html") #change minimum

Difference between 266 and 221 is 45
Beacon St / Mass Ave
MIT at Mass Ave / Amherst St
____________________________________
Difference between 221 and 266 is 45
MIT at Mass Ave / Amherst St
Beacon St / Mass Ave
____________________________________
Difference between 107 and 194 is 87
MIT Stata Center at Vassar St / Main St
MIT Vassar St
____________________________________
Difference between 63 and 137 is 74
Davis Square
Teele Square at 239 Holland St
____________________________________
Difference between 137 and 63 is 74
Teele Square at 239 Holland St
Davis Square
____________________________________
Difference between 194 and 107 is 87
MIT Vassar St
MIT Stata Center at Vassar St / Main St
____________________________________


## TODO: 

### Easy:
-  Draw grid (thin black lines)
-  Test with colors instead of weight

### Medium difficult

-  Draw arcs instead of straight lines so that the two ways of a route are clearly separated
-  Analyze trips in the diagonal (that are currently ignored)


## A map of all trips in a month

I've tried with data from July (summer break for schools) and May (Spring semester for schools): it's a bit different but not very much. Maybe university students that take the break don't use much bikesharing, they either have their own bikes or use other means.

In [207]:
flow_map(od_matrix)

NameError: name 'od_matrix' is not defined

## And now showing where the dock stations are

In [157]:
# for debugging, let's draw the bike stations on the map

boston_map = folium.Map([(north_limit+south_limit)/2, (west_limit+east_limit)/2], zoom_start = 13, control_scale = True)

for index, row in stations.iterrows():
    folium.Marker(location=[row.lat, row.lon],icon=folium.Icon(color='green')).add_to(boston_map)
boston_map.save("flows_with_stations.html")
boston_map

## Morning trips only

In [ ]:
flow_map(od_morning,  minimum=10, file_to_save="boston_morning_flow_workday_may.html")


## Afternoon trips

In [ ]:
flow_map(od_afternoon, show = 5, file_to_save="boston_afternoon_flow_workday_may.html")


## Lunch-time trips

In [ ]:
flow_map(od_lunch, show = 8, file_to_save="boston_lunch_flow_workday_may.html")


#### Further Ideas

-  interactive system that 
    -  shows initially the grid with the major flows
    -  it lets you select (morning peak, noon, afternoon peak, late evening) and (workday vs. weekend)
    -  once you select a grid cell, it zooms in and shows the stations on that grid
    -  once you select a station, it shows the most common in and out flows for that station
    -  by selecting a grid cell, it shows the socioeconomic data and POIs for that cell 
    -  for each grid, the information is shown both graphically and in a text-based table
    
-  draw a graph of the distribution of trips per route. E.g., a bar graph in which Y is the # of trips and each point in the X axis is a route (Y sorted from left to right)

- Compute some descriptive statistics:
    - average trip length and duration, median, stadard deviation, min, max. - Leverage *pandas.pivot_table*
    - different regions of the city have different patterns?
    - different times of the day have different patterns?
    - is any day of the week in the working week days different?
    - can we use clustering algorithms to find these things automatically?
    
- Characterize the distribution of trips across the city
    - in the areas that concentrate a lot of trips, are there many stations? Is the capacity of stations appropriate?
        - this may identify busy areas that should have more infrastructure
    - in the areas that have a small number of trips, are there too many stations? Stations with large capacity? This may indicate an excess of infrastructure, waste of resources.
    - are the stations in the periphery (limits of the system) used heavily?
        - this might indicate that the system should be expanded geographically
    - how much dispersed are the trips? They are highly concentrated in one area? In a few areas? Or they a spread across many areas in different parts of the city?
        - For each city, we can compute the distribution of trips wrt the distance to city center. Comparing the distribution for different cities, we can measure how centralized the cities are. We can do the same, not with the geographical center but with the major bike hubs of the city.
    - Which proportion of the trips start or finish close to a subway station? And close to a bus stop?
    - Within working days, is there any significant change in the patterns?
    - Are there differences in lunch-time practices in different cities? Madrid (Siesta), US, Brazil, Europe?
    - In the major flows, the trips are evenly spaced throughout the day or they happen in small bursts. Do these major flows reflect a lack of public transportation?
    - How much pendular are the flows (characterizing commuting behavior) and how much not pendular it is (characterizing other types of use)
    
---

- **Overall Research Questions:** Given a city and its bikesharing trips, can I answer the following questions:
    -  Are the stations correctly distributed across the city? Or are there areas underserved and overserved? What are them?
    -  What are the major hubs for bikesharing where the infrastructure should be improved?
    -  What are the primary and secondary routes for bikesharing for which a good cycling infrastructure should exist? For example, segregated cycle tracks in primary routes and painted cycle lanes in secondary routes.
    
    